# 📘 Bookflix – Sprint 3: Sistema de Recomendación
**Autores:**  
- Daniel Fabián Rodríguez Lorenzo  
- Tatiana María Quintas Rodríguez  
- Miguel Ángel Seara Losada  
- David Simón Nóvoa  

**Objetivo:**  
Demostrar el sistema de recomendación usando el dataset de libros de Bookflix.  

explicacion de lo que vamos a hacer

esto seria para calcular para cada libro los libros similares, esto se guardaria en le DB y luego se recuperaria de la propia DB, el sistema de recomendación se ejecutaria previamente, no al buscar

Mirar en moovi explicaciones mas detalladas:
carga de datos (lo que ya sabemos)
Pasos de Preprocesado de texto:
-Tokenizacion (transforma un texto en un array de palabras)
-Eliminacion de stopwords (palabras sin significado, prefijos...)
-Stemizacion (eliminar variaciones de conjugacion, genero y numero)

Pasar a el texto preprocesado a bag of words (array que indica para cada palabra cuantas veces aparece)

Usar algoritmo TF-IDF para conseguir el bag of words con pesos

Calcular las distancias entre los textos
A menor distancia, mas similitud

# Cargar dataset

Primero cargamos el dataset data.csv en un DataFrame de pandas para poder manipularlo y analizarlo de manera eficiente.
(Nota : Primero es necesario cargar manualmente el archivo al notebook)
Mostramos una vista previa de los datos para comprobar que se han cargado correctamente.

In [5]:
import pandas as pd

df_og = pd.read_csv('data.csv')
df_og

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0
6806,9788185944609,8185944601,Secrets Of The Heart,NaN,Khalil Gibran,Mysticism,http://books.google.com/books/content?id=XcrVp...,NaN,1993.0,4.08,74.0,324.0
6807,9788445074879,8445074873,Fahrenheit 451,NaN,Ray Bradbury,Book burning,NaN,NaN,2004.0,3.98,186.0,5733.0
6808,9789027712059,9027712050,The Berlin Phenomenology,NaN,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0


# Preprocesado del dataset

En esta celda se preprocesa el texto almacenado en el campo "description" y se almacena en una nueva columna del DataFrame con nombre "processed_desc".
El preprocesamiento incluye los pasos indicados previamente: tokenización del texto (mediante la libreria *nltk* ), eliminación de stopwords (usando como referencia stopwords por defecto de la libreria *nltk* ) y stemmización (usando el algoritmo *PorterStemmer* )

Al finalizar el preprocesamiento, se muestran los datos para verificar que el proceso se ha realizado con exito, y se muestra la descripcion y el texto procesado del primer libro para comparar.

In [14]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

processed_desc = []

# Fill NaN values in 'description' column with empty strings
df_og['description'] = df_og['description'].fillna('')

for row in df_og.itertuples():
    text = word_tokenize(row[8]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    processed_desc.append(text)

df_preprocessed = df_og
df_preprocessed['processed_desc'] = processed_desc

df_preprocessed


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,processed_desc
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,a novel that reader critic eagerli anticip dec...
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,a new christma novel adapt acclaim play charl ...
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,volum two stephen donaldson acclaim second tri...
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,a memor mesmer heroin jennif brilliant beauti ...
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,lewi work natur love divid love four categori ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,thi collect timeless teach one greatest sage i...
6806,9788185944609,8185944601,Secrets Of The Heart,NaN,Khalil Gibran,Mysticism,http://books.google.com/books/content?id=XcrVp...,,1993.0,4.08,74.0,324.0,
6807,9788445074879,8445074873,Fahrenheit 451,NaN,Ray Bradbury,Book burning,NaN,,2004.0,3.98,186.0,5733.0,
6808,9789027712059,9027712050,The Berlin Phenomenology,NaN,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0,sinc three volum edit ofhegel philosophi subje...


In [15]:
df_preprocessed.iloc[0]['description']


'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [16]:
df_preprocessed.iloc[0]['processed_desc']


'a novel that reader critic eagerli anticip decad gilead astonishingli imagin stori remark live john ame preacher son preacher grandson matern patern preacher it 1956 gilead iowa toward end reverend ame life absorb record famili stori legaci young son never see grow haunt grandfath presenc john tell rift grandfath father elder angri visionari fought abolitionist caus son ardent pacifist he troubl prodig namesak jack john ame boughton best friend lost son return gilead search forgiv redempt told john ame joyou rambl voic find beauti humour truth smallest life detail gilead song celebr accept best worst world offer at heart tale sacr bond father son style stori set dazzl critic reader alik'